In [ ]:
rules1 = (
    "Rule1: if {P3} is true then {P2} is true.\n"
    "Rule2: if {R2} is true, then if {P1} is true then {Q1} is True.\n"
    "Rule3: {P1} is true and {R1} is true.\n"
    "Rule4: if {R1} is true then {S1} is true.\n"
)
rules2 = ( 
    #"Premise1: if {P} is true then {Q} is true.\n"
    "Premise2: if {R} is true then {S} is true.\n"
    "Premise3: {P} is true or {S} is not true.\n"
    "Premise4: if {Q} is true then {T} is true.\n"
)
commensence = (
    
    "facts: if {P} is true then {Q} is true.\n"
)
prompt1 = (
    "##Instruction:\n1. Please think about and understand the following logical reasoning rules.\n"
    "2. Generate a short real life story that includes sentences to illustrate the above rules, replacing the entities with real values. Do not include the entity labels like P, Q, R, S, T in the story\n"
    "3. Generate the followling complex reasoning question using the story and the rules.\n"
    "4. Output format is: {\"Context\":***,\"Question\":***} \n"
    "##Rules:\n"
    f"{rules1}"
    "##Question: If {P3} is true , then {Q1} and {S1} is true?\n"
    "##Output:"
)
prompt2 = (
    "##Instruction:\n1. Please understand the following logical premises.\n"
    "2. Generate common-sense knowledge that aligns with this logical relationship P->Q and is familiar to humans."
    "3. Replace the entities in the premises with real-world facts from a same field.\n"
    #"3. Generate a rich and naturally coherent narrative story using the generated facts, incorporating the following premises.\n"
    "4. Generate a complex logical reasoning question using the generated facts and common-sense knowledge, which needs multi-step reasoning\n"
    "5. Output format is: {\"Commensence\":***,\"Facts\":***,\"Question\":***} \n"
    f"Premises:{rules2}\n##Output:"
)
prompt3 = (
    "##Instruction:Write a multi-choice deductive question for the following article, which needs multi-step reasoning:\n"
    "##Article: Dr. Smith was reviewing the case of a new patient, John, who had come in with severe symptoms. According to the initial tests, John had a bacterial infection. Dr. Smith knew that if a patient had a bacterial infection, they would need antibiotics. Therefore, John was prescribed a course of antibiotics. Furthermore, John had a high fever. Dr. Smith understood that if a patient had a high fever, they needed to be hospitalized. So, John was admitted to the hospital for close monitoring and care. Dr. Smith also knew that if a patient needed antibiotics, they would recover in two weeks. Given that John had a bacterial infection, it was expected that with the right treatment, he would be well on his way to recovery in a fortnight. The medical team worked around the clock to ensure John's condition improved. As predicted, John's fever started to subside within a few days, and by the end of two weeks, he was on the path to full recovery.The medical staff was satisfied with the outcome, as it aligned perfectly with their expectations based on the premises they knew to be true. John was discharged from the hospital, feeling much better and grateful for the diligent care he received.\n"
    "##Output:"
)

In [17]:
#from vllm import LLM, SamplingParams
from openai import OpenAI
import json

openai_api_key = "sk-mRfbc5lSi96hWwmm7c3623E8B41c41Dc9cC4E40453029131"
openai_api_base = "https://open.xiaoai.one/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

prompt = prompt2
messages = [
    {"role": "system", 
     "content": "Complete the following tasks according to the instruction, only returning text in the specified output format"},
    {"role": "user", "content": prompt},
]
print(messages[0]["content"]+"\n"+prompt)
gen_params = {
    "max_tokens": 1024,
    "temperature": 0.9,
    "top_p": 0.9,
    "stream": False,
}
chat_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    **gen_params
)
output = chat_response.choices[0].message.content
# Define the file path
print(output)
file_path = "output.jsonl"
# Find the index of the first occurrence of 'a' in the output string
index = output.find('{')
# Check if 'a' is found in the output string
if index != -1:
    res = output[index:]
    print(res)
    with open(file_path, "a", encoding='utf-8') as file:
    # Write the output to the file
        file.write(res + "\n")





Complete the following tasks according to the instruction, only returning text in the specified output format
##Instruction:
1. Please understand the following logical premises.
2. Generate common-sense knowledge that aligns with this logical relationship P->Q and is familiar to humans.3. Replace the entities in the premises with real-world facts from a same field.
4. Generate a complex logical reasoning question using the generated facts and common-sense knowledge, which needs multi-step reasoning
5. Output format is: {"Commensence":***,"Facts":***,"Question":***} 
Premises:Premise2: if {R} is true then {S} is true.
Premise3: {P} is true or {S} is not true.
Premise4: if {Q} is true then {T} is true.

##Output:
{"Commensence":"If it is raining, the ground will be wet. If the ground is wet, people tend to carry umbrellas. If people carry umbrellas, they are likely prepared for rain.","Facts":"If it is raining, then the ground will be wet. It is either raining or the ground is not wet. I

In [1]:
#解析表达式，看它符合哪一条规则的前提形式
#利用符合的规则推导得到新的结论

#判断两个命题最外层的结构是否相同
#定义树结构表示命题，然后比较树结构是否相同
import random
import json
class TreeNode:
    def __init__(self, val):
        self.val = val   #整个的表达式字符串
        self.sym = None  #二元计算符号 或 命题表达式
        self.left = None  #子节点
        self.right = None #子节点
        
def preorder_traversal(prop:TreeNode) -> list:
    result = []
    if prop.left!=None and prop.right!=None:
        result.append("(")
    if prop.left:
        result.extend(preorder_traversal(prop.left))
    result.append(prop.sym)
    if prop.right:
        result.extend(preorder_traversal(prop.right))
    if prop.left!=None and prop.right!=None:
        result.append(")")
    return result
def is_valid_parentheses(s: str) -> bool:
        stack = []

    # 遍历字符串中的每个字符
        for char in s:
            if char == ')':
            # 如果遇到闭括号，检查栈顶是否有匹配的开括号
                top_element = stack.pop() if stack else '#'
                if top_element != '(':
                    return False
            elif char == '(':
             # 如果是开括号，将其压入栈中
                stack.append(char)
        # 检查栈是否为空，空表示所有括号都匹配
        return not stack
def find_first_symbol(s:str) -> str:
        stack = []
        for i in range(len(s)):
            if s[i] == '(':
                stack.append(s[i])
            elif s[i] == ')':
                stack.pop()
            elif not stack and s[i] in ['>','&','|']:
                return i
            elif not stack and s[i] in ['~']:
                return i
        return -1    
def create_tree(s:str):
    prop = TreeNode(s)
    l = len(s)
    if s[0] == '(' and s[-1] == ')':
        if is_valid_parentheses(s[1:l-1]):
            s = s[1:l-1]
    index = find_first_symbol(s)
    if index == -1:
        prop.sym = s
        return prop
    prop.sym = s[index]
    parts = [s[:index],s[index+1:]]
    if parts[0]:
        prop.left = create_tree(parts[0])
    # Create the right child node
    if parts[1]:
        prop.right = create_tree(parts[1])
    return prop

class Proposition:
    def __init__(self,s:str):
        self.val = s
        self.stt = create_tree(s)
    def replace(self,o_node:TreeNode,t_node:TreeNode):
        new_p = self
        def get_leaf_nodes(root: TreeNode,num:int):
            if root is None:
                return
            if root.left is None and root.right is None:
                root.sym+=str(num)
                root.val = root.sym
            else:
                get_leaf_nodes(root.left,num)
                get_leaf_nodes(root.right,num)
            return root
        #new_p.stt
    def p_list(self):
        l = preorder_traversal(self.stt)
        l = [i for i in l if i not in ['(',')','>','&','|','~']]
        return l

def format_equal(p1:TreeNode,p2:TreeNode):
    if p1==None and p2==None:
        return True
    elif (p1==None and p2!=None) or p2==None and p1!=None:
        return False
    elif  p2.sym not in ['>','&','|','~']:
        return True
    elif p1.sym == p2.sym:
        return format_equal(p1.left,p2.left) and format_equal(p1.right,p2.right)
    else:
        return False
print(format_equal(create_tree("(r&q)>s"),create_tree("p>q")))
#给定规则名称，和初始的部分前提，构造出符合要求的全部前提条件
#待补充
def add_num_proposition(p:Proposition,num):
    prop = create_tree(p.val)
    def get_leaf_nodes(root: TreeNode,num:int):
        if root is None:
            return
        if root.left is None and root.right is None:
            root.sym+=str(num)
            root.val = root.sym
        else:
            get_leaf_nodes(root.left,num)
            get_leaf_nodes(root.right,num)
    get_leaf_nodes(prop,num)
    s_list = preorder_traversal(prop)
    s = ''.join(s_list)
    new_p = Proposition(s)
    return new_p

    leaf_nodes = get_leaf_nodes(p.stt)
#def replace_prop(prop:Proposition,o_node:TreeNode,t_node:TreeNode)->Proposition:
print(add_num_proposition(Proposition("p>q"),1).val)

Rules = [
    {"name":"MP","Premise":[Proposition("p>q"),Proposition("p")],"Conclusion":Proposition("q")},
    {"name":"DS","Premise":[Proposition("p|q"),Proposition("~p")],"Conclusion":Proposition("q")},
    {"name":"MT","Premise":[Proposition("p>q"),Proposition("~q")],"Conclusion":Proposition("~p")},
    {"name":"HS","Premise":[Proposition("p>q"),Proposition("q>r")],"Conclusion":Proposition("p>r")},
    {"name":"CD","Premise":[Proposition("p>q"),Proposition("r>s"),Proposition("p|r")],"Conclusion":Proposition("q|s")},
    {"name":"AD","Premise":[Proposition("p>q"),Proposition("r>s"),Proposition("p&r")],"Conclusion":Proposition("q&s")},
    {"name":"DD","Premise":[Proposition("p>q"),Proposition("r>s"),Proposition("~q|~s")],"Conclusion":Proposition("~p|~r")},
    {"name":"BD","Premise":[Proposition("p>q"),Proposition("r>s"),Proposition("p|~s")],"Conclusion":Proposition("q|~r")},
    #{"name":"CT","Premise":[Proposition("p>q"),Proposition("r>s"),Proposition("q|~r")],"Conclusion":Proposition("~p|s")},
    {"name":"CO","Premise":[Proposition("p>q"),Proposition("p>r")],"Conclusion":Proposition("p>(q&r)")},
    {"name":"IM","Premise":[Proposition("p>(q>r)")],"Conclusion":Proposition("(p&q)>r")},
    {"name":"DMT","Premise":[Proposition("~(p&q)")],"Conclusion":Proposition("~p|~q")},
    {"name":"DMS","Premise":[Proposition("~(p|q)")],"Conclusion":Proposition("~p&~q")},
    {"name":"MI","Premise":[Proposition("p>q")],"Conclusion":Proposition("~p|q")},
]
# 目前一个关键的问题在于，这样简单的拼接，使得最后的逻辑命题可能会很复杂，不是很巧妙

def pairwise(p1:TreeNode,p2:TreeNode):
    res = {}
    if p2 == None:
        return res
    elif p1.sym in ['>','&','|','~'] and p2.sym in ['>','&','|','~'] and p1.sym == p2.sym:
        res.update(pairwise(p1.left,p2.left))
        res.update(pairwise(p1.right,p2.right))
    elif  p2.sym not in ['>','&','|','~']:
        res[p2.sym]=p1.val
    return res
def change_proposition(p:Proposition,change:dict):
    prop = create_tree(p.val)
    def change_node(root: TreeNode,change:dict):
        if root is None:
            return
        if root.sym in ['>','&','|','~']:
            root.left=change_node(root.left,change)
            root.right=change_node(root.right,change)
        else:
            if root.sym in change:
                root.sym = change[root.sym]
                root.val = root.sym
        return root
    new_prop=change_node(prop,change)
    s_list = preorder_traversal(new_prop)
    s = ''.join(s_list)
    return s 
print(pairwise(create_tree("(r&s)>q"),create_tree("p>q")))
def reversed_inference(conclusion:Proposition,num):
    #prop = [add_num_proposition(i,num) for i in Rules[rule]["Premise"] ]
    #conc = add_num_proposition(Rules[rule]["Conclusion"],num)
    #name = Rules[rule]["name"]
    r_prop = []
    r_conc = []
    if conclusion == None:
        rule = random.choice(Rules)
        prop = [add_num_proposition(i,num) for i in rule["Premise"] ]
        name = rule["name"]
        conc = add_num_proposition(rule["Conclusion"],num)
        r_prop = [p.val for p in prop]
        r_conc = [conc.val]
    else:
        rule_list = []
        for i in Rules:
            if format_equal(conclusion.stt,i["Conclusion"].stt):
                rule_list.append(i)
        if len(rule_list) == 0:
            print("No rule found")
        rule = random.choice(rule_list)
        name = rule["name"]
        rep = pairwise(conclusion.stt,add_num_proposition(rule["Conclusion"],num).stt)
        print(rep)
        prop = [add_num_proposition(i,num) for i in rule["Premise"] ]
        
        #if format_equal(conclusion,Rules[rule]["Conclusion"]):
        #    pass
        #else:
        r_prop = [change_proposition(p,rep) for p in prop]
        r_conc = [conclusion.val]
    return {"name":name,"Premise":r_prop,"Conclusion":r_conc}

print(Rules[7]["Conclusion"].p_list())
'''
for rule in Rules:
    print("-------------")
    print("name:",rule["name"])
    premise = []
    for p in rule["Premise"]:
        premise.extend(preorder_traversal(p.stt))
        if p != rule["Premise"][-1]:
            premise.append("&")
    print("Premise:",premise)
    print("Conclusion:",preorder_traversal(rule["Conclusion"].stt))
'''
def generate_mulitlogic(num:int,deep:int):
    i = 0
    result = []
    c = None
    while i < num:
        i+=1
        #rule = random.randint(0,10)
        #print(rule)
        one_hop=reversed_inference(c,i)
        result.append(one_hop)
        c = Proposition(random.choice(one_hop["Premise"]))
    return result
result = []
for i in range(2,6):
    for j in range(1,5):
        chain = generate_mulitlogic(i,0)
        chain.reverse()
        result.append({
            "hop_num":i,
            "chain":chain
        })
        # Define the file path
file_path = "chain.jsonl"
        # Write the result list to the jsonl file
with open(file_path, "a") as file:
    for item in result:
        json.dump(item, file)
        file.write("\n")
#print(one_inference(None,3,1))


True
(p1>q1)
{'p': '(r&s)', 'q': 'q'}
['q', 'r']
{'q2': '(p1>q1)'}
{'q2': '(p1>q1)'}
{'p2': 'r1', 'r2': 's1'}
{'q2': '(p1>r1)'}
{'p2': 'p1', 'r2': 'q1'}
{'q3': '(p1>q2)'}
{'q2': '(r1>s1)'}
{'p3': 'p2'}
{'q2': '(p1>(q1>r1))'}
{'q3': '(p2>(p1>(q1>r1)))'}
{'q2': '(p1>q1)'}
{'q3': 'p2'}
{'p2': 'p1', 'r2': 'q1'}
{'q3': '(p1>q2)'}
{'q4': '(p3>(p1>q2))'}
{'q2': '(p1|~s1)'}
{'p3': 'p2', 'r3': '(p1|~s1)'}
{'q4': '(p2>q3)'}
{'q2': '(p1>r1)'}
{'q3': '(p2>(p1>r1))'}
{'q4': 'p3'}
{'p2': '(p1|q1)'}
{'p3': '(p1|q1)', 'r3': 'q2'}
{'q4': '(q3>q2)'}
{'q2': '(p1>q1)'}
{'q3': 'p2'}
{'q4': 'p3'}
{'p5': 'p4'}
{'q2': '~(p1|q1)'}
{'q3': '(p2|~(p1|q1))'}
{'q4': '~p3'}
{'q5': '~p4'}
{'p2': 'p1', 'r2': 'q1'}
{'q3': '(q2>q1)'}
{'q4': '(p3>(q2>q1))'}
{'q5': 'p4'}
{'q2': '(p1|~s1)'}
{'q3': '(p2|(p1|~s1))'}
{'q4': '~p3'}
{'q5': '~p4'}


In [91]:
import re

def logical_to_natural(expression):
    def translate(expr, tag):
        expr = expr.strip()
        if tag == 1:
            return expr 
        else:
            if expr[0] == '(' and expr[-1] == ')':
                expr = expr[1:-1]
            if '>' in expr:
                left, right = map(str.strip, expr.split('>', 1))
                return f"\"if {translate(left,0)}, then {translate(right,0)}\""
            elif '&' in expr:
                parts = map(str.strip, expr.split('&'))
                return "\"" + " and ".join([f"{translate(part,0)}" for part in parts]) + "\""
            elif '|' in expr:
                parts = map(str.strip, expr.split('|'))
                return "\"" + ", or ".join([f"{translate(part,0)}" for part in parts]) + "\""
            elif '~' in expr:
                inner = expr[1:].strip()
                return f"{translate(inner,1)} is not true"
            else:
                return expr + " is true"

    def parse_nested(expr):
        while '(' in expr:
            # Find the deepest nested expression
            # print(expr)
            nested = re.findall(r'\([^()]+\)', expr)
            for sub_expr in nested:
                # print(sub_expr)
                translated = translate(sub_expr,0)
                expr = expr.replace(f'{sub_expr}', translated)
        return translate(expr,0)

    return parse_nested(expression) + "."

In [94]:
#cc是一个计数id，用来调试；
#validation是判断句子是否合法的哨兵
#rule_result设计用来存储最终输出的所有Rules，可以输出到文件看看
cc = 0
validation = 1
rule_result = []
def strcmp(s1, s2):
    i = 0
    while i < len(s1) and i < len(s2):
        if s1[i] == s2[i]:
            i += 1  
        else:
            return ord(s1[i]) - ord(s2[i])
    return len(s1) - len(s2)

def is_depth_valid(rules : str) -> bool :
    stack = []
    # 遍历字符串中的每个字符
    for char in rules:
        if char == ')':
        # 如果遇到闭括号，检查栈顶是否有匹配的开括号
            top_element = stack.pop() if stack else '#'
            if top_element != '(' or len(stack) > 2:
                return False
        elif char == '(':
            # 如果是开括号，将其压入栈中
            stack.append(char)
    # 检查栈是否为空，空表示所有括号都匹配
    return not stack
        
for item in result:
    # if cc != 14:
    #     cc += 1
    #     continue
    chain = item["chain"]
    rules = []
    conclusion = ""
    for step in chain:
        premise = step["Premise"]
        for element in premise:
            validation = is_depth_valid(element)
            if not validation:
                break
            if strcmp(element,conclusion) != 0 :
                rule = logical_to_natural(element)
                rules.append(rule)
                print(rule)
        if not validation:
            break
        conclusion = step["Conclusion"][0]
    if not validation:
        print(f"invalid,id:{cc}")
        cc += 1
        continue
    rule = logical_to_natural(conclusion)
    rules.append(rule)
    print(rule)
    rule_result.append({
        "id": cc,
        "chain": item["chain"],
        "rules": rules
    })
    cc += 1

with open('./rules.jsonl', 'w', encoding='utf-8') as f:
    for record in rule_result:
        json.dump(record, f, ensure_ascii=False)
        f.write("\n")

print("Rules have been written to ./rules.jsonl")

rules1 = (
    "Rule1: if {P3} is true then {P2} is true.\n"
    "Rule2: if {R2} is true, then if {P1} is true then {Q1} is True.\n"
    "Rule3: {P1} is true and {R1} is true.\n"
    "Rule4: if {R1} is true then {S1} is true.\n"
)
rules2 = ( 
    #"Premise1: if {P} is true then {Q} is true.\n"
    "Premise2: if {R} is true then {S} is true.\n"
    "Premise3: {P} is true or {S} is not true.\n"
    "Premise4: if {Q} is true then {T} is true.\n"
)
commensence = (
    
    "facts: if {P} is true then {Q} is true.\n"
)
prompt1 = (
    "##Instruction:\n1. Please think about and understand the following logical reasoning rules.\n"
    "2. Generate a short real life story that includes sentences to illustrate the above rules, replacing the entities with real values. Do not include the entity labels like P, Q, R, S, T in the story\n"
    "3. Generate the followling complex reasoning question using the story and the rules.\n"
    "4. Output format is: {\"Context\":***,\"Question\":***} \n"
    "##Rules:\n"
    f"{rules1}"
    "##Question: If {P3} is true , then {Q1} and {S1} is true?\n"
    "##Output:"
)
prompt2 = (
    "##Instruction:\n1. Please understand the following logical premises.\n"
    "2. Generate common-sense knowledge that aligns with this logical relationship P->Q and is familiar to humans."
    "3. Replace the entities in the premises with real-world facts from a same field.\n"
    #"3. Generate a rich and naturally coherent narrative story using the generated facts, incorporating the following premises.\n"
    "4. Generate a complex logical reasoning question using the generated facts and common-sense knowledge, which needs multi-step reasoning\n"
    "5. Output format is: {\"Commensence\":***,\"Facts\":***,\"Question\":***} \n"
    f"Premises:{rules2}\n##Output:"
)
prompt3 = (
    "##Instruction:Write a multi-choice deductive question for the following article, which needs multi-step reasoning:\n"
    "##Article: Dr. Smith was reviewing the case of a new patient, John, who had come in with severe symptoms. According to the initial tests, John had a bacterial infection. Dr. Smith knew that if a patient had a bacterial infection, they would need antibiotics. Therefore, John was prescribed a course of antibiotics. Furthermore, John had a high fever. Dr. Smith understood that if a patient had a high fever, they needed to be hospitalized. So, John was admitted to the hospital for close monitoring and care. Dr. Smith also knew that if a patient needed antibiotics, they would recover in two weeks. Given that John had a bacterial infection, it was expected that with the right treatment, he would be well on his way to recovery in a fortnight. The medical team worked around the clock to ensure John's condition improved. As predicted, John's fever started to subside within a few days, and by the end of two weeks, he was on the path to full recovery.The medical staff was satisfied with the outcome, as it aligned perfectly with their expectations based on the premises they knew to be true. John was discharged from the hospital, feeling much better and grateful for the diligent care he received.\n"
    "##Output:"
)

"p2 is true, or "if p1 is true, then q1 is true" is true" is true.
p2 is not true.
p1 is true.
q1 is true.
"p2 is true, or "if p1 is true, then q1 is true" is true" is true.
p2 is not true.
"if r1 is true, then s1 is true" is true.
"q1 is true, or s1 is not true" is not true.
"p1 is true, or r1 is not true" is not true.
"if r1 is true, then q2 is true" is true.
"if q2 is true, then s1 is true" is true.
"if p1 is true, then q1 is true" is true.
"p1 is true, or s1 is not true" is true.
"q1 is true, or r1 is not true" is true.
"p2 is true, or "if p1 is true, then r1 is true" is true" is true.
p2 is not true.
"if p1 is true, then q1 is true" is true.
"if p1 is true, then "q1 is true and r1 is true" is true" is true.
"p3 is true, or "if p1 is true, then q2 is true" is true" is true.
p3 is not true.
"if q2 is true, then q1 is true" is true.
"if r1 is true, then s1 is true" is true.
"p1 is true, or s1 is not true" is true.
"q1 is true, or r1 is not true" is true.
"if p2 is true, then q3 is tr

In [ ]:
import re

def logical_to_natural(expression):
    def translate(expr):
        expr = expr.strip()
        if '>' in expr:
            left, right = map(str.strip, expr.split('>', 1))
            return f"if {translate(left)} is true, then {translate(right)} is true."
        elif '&' in expr:
            parts = map(str.strip, expr.split('&'))
            return " and ".join([f"{translate(part)}" for part in parts])
        elif '|' in expr:
            parts = map(str.strip, expr.split('|'))
            return " or ".join([f"{translate(part)}" for part in parts])
        elif '~' in expr:
            inner = expr[1:].strip()
            return f"it is not true that {translate(inner)}"
        else:
            return expr

    def parse_nested(expr):
        while '(' in expr:
            # Find the deepest nested expression
            # print(expr)
            nested = re.findall(r'\([^()]+\)', expr)
            for sub_expr in nested:
                print(sub_expr)
                translated = translate(sub_expr[1:-1])
                expr = expr.replace(f'{sub_expr}', translated)
        return translate(expr)

    return parse_nested(expression)

In [41]:
# Example
expression = "(p1>(q1&r1))"
natural_language = logical_to_natural(expression)
print(natural_language)

(q1&r1)
(p1>q1 and r1)
If p1 is true, then q1 and r1 is true.


In [42]:
s = logical_to_natural("(p2|(p1>q1))")
print (s)

(p1>q1)
(p2|If p1 is true, then q1 is true.)
p2 or If p1 is true, then q1 is true.
